# 5-1. **Topic Modeling**

In [ ]:
!pip install -q nltk
!pip install -q konlpy
!pip install -q gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 23.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!wget https://raw.githubusercontent.com/kimtwan/NLP_lecture/master/data/MPB_minutes.tsv

--2023-10-17 00:18:38--  https://raw.githubusercontent.com/kimtwan/NLP_lecture/master/data/MPB_minutes.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15195720 (14M) [text/plain]
Saving to: ‘MPB_minutes.tsv’

MPB_minutes.tsv     100%[===================>]  14.49M  37.3MB/s    in 0.4s    

2023-10-17 00:18:39 (37.3 MB/s) - ‘MPB_minutes.tsv’ saved [15195720/15195720]



In [ ]:
# load Monetary Policy Board minutes text
data = pd.read_csv('MPB_minutes.tsv', encoding='utf-8', sep='\t', parse_dates=['날짜'], index_col='날짜')
# minutes after 2020
data = data.loc['2020-01-01':]
data.head()

<ipython-input-4-be31f9254d68>:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  data = data.loc['2020-01-01':]


,회차,의사록
날짜,,
2020-05-28,2020년도 제12차,2020년도 제12차\n금융통화위원회 의사록\n한 국 은 행\n1. 일 자 2020...
2020-07-16,2020년도 제15차,2020년도 제15차\n금융통화위원회 의사록\n한 국 은 행\n1. 일 자 2020...
2020-08-27,2020년도 제19차,2020년도 제19차\n금융통화위원회 의사록\n한 국 은 행\n1. 일 자 2020...
2020-01-17,2020년도 제1차,2020년도 제1차\n금융통화위원회 의사록\n한 국 은 행\n1. 일 자 2020년...
2020-10-14,2020년도 제22차,2020년도 제22차\n금융통화위원회 의사록\n한 국 은 행\n1. 일 자 2020...


In [ ]:
from konlpy.tag import Kkma
from nltk import sent_tokenize
from tqdm import notebook

In [ ]:
kkma = Kkma()
corpus = data['의사록'].tolist()

corpus_nouns = []
for i in notebook.tqdm(range(len(corpus))):
    # separate the minutes into sentences
    sentences = sent_tokenize(corpus[i])
    for sentence in sentences:
        # extract nouns
        nouns = kkma.nouns(sentence)
        # includes only words longer than 2 characters
        nouns = [noun for noun in nouns if len(noun) > 1]
        corpus_nouns.append(nouns)

  0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
for i in range(0, 5):
    print(corpus_nouns[i])

['2020', '2020년', '12', '12차', '금융', '금융통화위원회', '통화', '위원회', '의사록']
['2020', '2020년', '5월', '28', '28일']
['금융', '금융통화위원회', '통화', '위원회', '회의실']
['출석', '출석위원', '위원']
['결석', '결석위원', '위원']


In [ ]:
# topic analysis using LDA
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

In [ ]:
# convert sentences into a form that can be used with gensim LDA
dictionary = Dictionary(corpus_nouns)
corpus = [dictionary.doc2bow(text) for text in corpus_nouns]

In [ ]:
# set number of topics
num_topics = 10

# run the model
lda = LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

In [ ]:
# print topics
topics = lda.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.030*"물가" + 0.029*"상승" + 0.024*"압력" + 0.023*"인플레이션" + 0.021*"통화" + 0.019*"정책" + 0.019*"금리" + 0.016*"은행" + 0.015*"방향" + 0.014*"통화정책방향"')
(1, '0.029*"대출" + 0.028*"가계" + 0.020*"금리" + 0.019*"상승" + 0.015*"가계대출" + 0.014*"주택" + 0.014*"증가" + 0.012*"가격" + 0.011*"지속" + 0.011*"신용"')
(2, '0.019*"외환" + 0.016*"차질" + 0.014*"국제" + 0.013*"경로" + 0.013*"리스크" + 0.013*"경제" + 0.012*"공급" + 0.012*"요인" + 0.011*"공급차질" + 0.011*"물가"')
(3, '0.024*"전망" + 0.018*"대비" + 0.017*"금년" + 0.016*"전월" + 0.016*"경상" + 0.014*"자본" + 0.013*"규모" + 0.013*"증가" + 0.013*"흑자" + 0.012*"확대"')
(4, '0.049*"관련" + 0.043*"관련부서" + 0.042*"부서" + 0.027*"물가" + 0.024*"답변" + 0.024*"위원" + 0.013*"필요" + 0.011*"언급" + 0.011*"인플레이션" + 0.010*"상승률"')
(5, '0.051*"금리" + 0.039*"인상" + 0.032*"기준" + 0.029*"기준금리" + 0.019*"위원" + 0.017*"수준" + 0.017*"일부" + 0.013*"상승" + 0.011*"회의" + 0.010*"금리인상"')
(6, '0.025*"자금" + 0.016*"투자" + 0.016*"증가" + 0.011*"외국인" + 0.011*"기업" + 0.011*"국내" + 0.010*"채권" + 0.009*"관련" + 0.009*"주식" + 0.009*"유입"')
(7, '0.018*"주체" + 0.018*"경제주체" +